In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import gc
import math

##  function Extract_NER():   Extract NER from receiving text list

In [3]:
def Extract_NER(ls_textRow):
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
    
    # 載入 自訂詞典
    User_Dict = {}
    with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
        for udic in UDicts:
            udWord = udic.strip().split(" ")
            if len(udWord) > 1:
                User_Dict[udWord[0]] = udWord[1]
            else:
                User_Dict[udWord[0]] = 10    # 未給定權重者一律賦予預設值 10                
    dictionary = construct_dictionary(User_Dict)
 
    ws = WS("./data")
    pos = POS("./data")
    ner = NER("./data")
    
    #  ls_segRow = ws(ls_textRow) ------------   without User Dictionary
    ls_segRow = ws(ls_textRow, 
                   sentence_segmentation=True,
                   segment_delimiter_set = {",", "。", ":", "?", "!", ";", "、"},
                   coerce_dictionary = dictionary)
    #print("ls_segRow = ", ls_segRow)
    #print("-----------------------------")
    
    del ws
    del WS
    gc.collect()
    ls_posRow = pos(ls_segRow)
    del pos
    del POS
    gc.collect()
    ls_nerRow = ner(ls_segRow,ls_posRow)
    del ner
    del NER
    gc.collect()
    #print("ls_nerRow = ", ls_nerRow)
    #print("-----------------------------")
    return ls_nerRow

## 1. Read data from csv or xlsx file

In [4]:
#讀檔
df_file = pd.read_csv(r"KCC Data/all-news(11554).csv")
print(df_file.columns)
print("-----------------------------------------------------------")
df_file.head(3)

Index(['序號', '日期', '報別', '內容'], dtype='object')
-----------------------------------------------------------


,序號,日期,報別,內容
0,1,2016/4/18,臺灣時報,馬英九總統昨天上午視察衛武營藝術文化中心，文化部長洪孟啟、工程會副主委顏久榮及高雄市副市長吳...
1,2,2016/4/18,臺灣時報,日本九州熊本縣接連遭強震襲擊，台灣分別捐贈熊本縣政府和日本政府款項，高雄市長陳菊、台南市 ...
2,3,2016/4/18,臺灣時報,壽山動物園為增進動物飼養福祉及保育工作，特設立「高雄市壽山動物園認養辦法」設立動物認養專戶...


## 2.  Main function:
### 2.1  Pass text data to function "Extract_NER" by row
### 2.2  Integrate every 10 NER return rows into one data frame  
### 2.3  Write frame into small csv after droping duplicate, locating class, filtering short NER, sorting by class.. 

In [4]:
#逐筆傳遞欄位 "text"
df_text = df_file["內容"]
print("內容 欄位筆數 = ",len(df_text))
print("------------------------------------------")
ls_class = list()
ls_ner = list()
fileCunt = 0
rowCunt = 0
for i in range(len(df_text)):
    rowCunt += 1
    ls_dfRow = [df_text[i]]
    # print("ls_dfRow = ", ls_dfRow)
    print("Row No = ",str(i))
    entity_list = Extract_NER(ls_dfRow)
    for j in range(len(entity_list[0])):
        seg = entity_list[0].pop()
        # print("seg = ",seg)
        ls_class.append(seg[2])
        ls_ner.append(seg[3])
    #print("===========================================")
    #print("ls_class = ",ls_class)
    #print("ls_ner = ",ls_ner)
    # if i == 5 or rowCunt == 2:
    if i == (len(df_text)-1) or rowCunt == 10:
        fileCunt += 1
        extract_df = pd.DataFrame({'Class':ls_class, 'NER':ls_ner})
        deDup_df = extract_df.drop_duplicates().copy()
        del extract_df
        cate_df = deDup_df.loc[deDup_df['Class'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE','NORP'])].copy()
        del deDup_df
        filt_df = cate_df[cate_df['NER'].map(len) >= 2].copy()
        del cate_df
        NER_df = filt_df.sort_values(by=['Class']).copy()
        del filt_df
        #print("__________________________________________________________")  
        #print(NER_df)
        print("fileCunt No = ",str(fileCunt))
        # NER_df.to_csv(r"News Data\News NER Out\\NewsNER870_" + str(fileCunt)+ ".csv")
        NER_df.to_csv(r"Ner_end" + str(fileCunt)+ ".csv")
        rowCunt = 0
print("=====================================================================")
print("Totle NER Files = ",fileCunt)


內容 欄位筆數 =  11554
------------------------------------------
Row No =  0


C:\Users\yangs\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\yangs\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  1
Row No =  2
Row No =  3
Row No =  4
Row No =  5
Row No =  6
Row No =  7
Row No =  8
Row No =  9
fileCunt No =  1
Row No =  10


C:\Users\yangs\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\yangs\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  11
Row No =  12
Row No =  13
Row No =  14
Row No =  15
Row No =  16
Row No =  17
Row No =  18
Row No =  19
fileCunt No =  2
Row No =  20


...

C:\Users\yangs\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\yangs\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  581
Row No =  582
Row No =  583
Row No =  584
Row No =  585
Row No =  586
Row No =  587
Row No =  588
Row No =  589
fileCunt No =  59
Row No =  590


C:\Users\yangs\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\yangs\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  591
Row No =  592
Row No =  593
Row No =  594
Row No =  595
Row No =  596
Row No =  597


KeyboardInterrupt: 

## 3. Integrate all csv files into single data frame

In [5]:
import glob
# import pandas as pd

# path = r'News Data\News NER OUT'
path = r'nerout'
#allFiles = glob.glob("{}/*.csv".format(path))
allFiles = glob.glob("./*.csv")

li = []
for filename in allFiles:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)
    print(li) 

nerFrame = pd.concat(li, axis=0, ignore_index=True)
print(len(nerFrame))
nerFrame.groupby('Class').count()

[      Class    NER
172   EVENT   二次大戰
57      GPE   中華民國
60      GPE     熊本
71      GPE     台中
79      GPE  九州熊本縣
..      ...    ...
281  PERSON     項羽
285  PERSON    孫翠鳳
294  PERSON    王永在
147  PERSON    蔡復進
350  PERSON    劉景寬

[124 rows x 2 columns]]
[      Class    NER
172   EVENT   二次大戰
57      GPE   中華民國
60      GPE     熊本
71      GPE     台中
79      GPE  九州熊本縣
..      ...    ...
281  PERSON     項羽
285  PERSON    孫翠鳳
294  PERSON    王永在
147  PERSON    蔡復進
350  PERSON    劉景寬

[124 rows x 2 columns],        Class      NER
1525   EVENT      五二０
2335   EVENT   二次世界大戰
2322   EVENT  全國黨代表大會
2534   EVENT       扁案
172    EVENT     二次大戰
...      ...      ...
996   PERSON      邱議瑩
994   PERSON      李永得
974   PERSON     陳菊市長
951   PERSON      陳麗娜
3698  PERSON    議長康裕成

[832 rows x 2 columns]]
[      Class    NER
172   EVENT   二次大戰
57      GPE   中華民國
60      GPE     熊本
71      GPE     台中
79      GPE  九州熊本縣
..      ...    ...
281  PERSON     項羽
285  PERSON    孫翠鳳
294  PERSON    王永在
147  PERSON 

,NER,page_id,page_name,page_category,message,story,type,permalink,created_time_gmt,created_time_taipei,...,link_name,link_description,like_count,haha_count,love_count,angry_count,sad_count,wow_count,comment_count,share_count
Class,,,,,,,,,,,,,,,,,,,,,
EVENT,5554,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GPE,21326,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LAW,1154,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LOC,8643,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NORP,2740,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ORG,57372,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PERSON,42676,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 4. Drop duplicates, group by class and write into csv file 

In [6]:
ddFrame = nerFrame.drop_duplicates().copy()
print(len(ddFrame))
ddFrame.groupby('Class').count()

31839


,NER,page_id,page_name,page_category,message,story,type,permalink,created_time_gmt,created_time_taipei,...,link_name,link_description,like_count,haha_count,love_count,angry_count,sad_count,wow_count,comment_count,share_count
Class,,,,,,,,,,,,,,,,,,,,,
EVENT,203,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GPE,633,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LAW,35,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LOC,293,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NORP,86,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ORG,1751,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PERSON,1406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# ddFrame.sort_values(by=['Class']).to_csv(r"News Data\News NER OUT\\NewsNER.csv")
ddFrame.sort_values(by=['Class']).to_csv(r"nerout")